# Chapter 2. Automating Tasks Made Easy
### Preparing a task (p. 47)
Command line program to multiply two numbers: `ch02-prepare-task.py`

In [3]:
import argparse

def main(n1,n2):
    result = n1 * n2
    print(f'result is {result}') # f-string with defined vars

if __name__ == '__main__':
    parser = argparse.ArgumentParser()
    parser.add_argument('-n1', type=int, help='num 1', default=1)    
    parser.add_argument('-n2', type=int, help='num 2', default=1)
    args = parser.parse_args()
    main(args.n1, args.n2)

usage: ipykernel_launcher.py [-h] [-n1 N1] [-n2 N2]
ipykernel_launcher.py: error: unrecognized arguments: --ip=127.0.0.1 --stdin=9003 --control=9001 --hb=9000 --Session.signature_scheme="hmac-sha256" --Session.key=b"6379f790-8a5d-486a-9e7f-3f4330f012eb" --shell=9002 --transport="tcp" --iopub=9004 --f=c:\Users\YURIS\AppData\Roaming\jupyter\runtime\kernel-v2-18872gV0PkcXhyurt.json


SystemExit: 2

c:\usr\yuri\python\py-autoCookBook\.venv\lib\site-packages\IPython\core\interactiveshell.py:3406: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)


in the book, `config.ini` processing is performed after parsing arguments and overrides the latter. This is counterintuitive, need to look into `argparse` module to see if `given` could be checked. Here is `config.ini`
```
[ARGUMENTS]
n1=5
n2=7
```

In [ ]:
import configparser

parser.add_argument('-c', '--config', type=argparse.FileType('r'), help='config file')

if args.config:
    config = configparser.ConfigParser()
    config.read_file(args.config)
    args.n1 = int(config['ARGUMENTS']['n1'])
    args.n2 = int(config['ARGUMENTS']['n2'])

In [ ]:
#adding output file
import sys

# maybe 'a' ?
parser.add_argument('-o', dest='output',type=argparse.FileType('w'),default=sys.stdout,help='output file')

main(args.n1,args.n2,args.output)
def main(n1,n2,output):
    print(f'{n1*n2}', file=output)


__[configparser doc](https://docs.python.org/3/library/configparser.html)__ <br>
More powerfult is __[YAML](https://learn.getgrav.org/advanced/yaml)__ <br>
Add `PyYAML==5.3` to `pip install -r requiremets.txt` and define `config.yaml`:
```
ARGUMENTS:
  n1: 7
  n2: 4
```

In [ ]:
import yaml
config = yaml.load(args.config, Loader= yaml.FullLoader)
# No need to transform values
args.n1 = config['ARGUMENTS']['n1']
args.n2 = config['ARGUMENTS']['n2']

### Setting up a cron job (p.53)
Script `ch02-cron.py` (unix only):

In [ ]:
import argparse
import sys
from datetime import datetime
import configparser

def main(n1,n2,out):
    print(f'[{datetime.utcnow().isoformat()}] result of {n1} * {n2} is {n1*n2}', file=out)

if __name__ == '__main__':
    # ArgumentDefaultsHelpFormatter -- gives defaults with -h
    parser = argparse.ArgumentParser(formatter_class=argparse.ArgumentDefaultsHelpFormatter)
    parser.add_argument('-c', '--config', type=argparse.FileType('r'), help='config file',default='/etc/automate.ini')
    parser.add_argument('-o',help='output', default=sys.stdout,  dest='output', type=argparse.FileType('a'))
    args = parser.parse_args()
    if args.config:
        config = configparser.ConfigParser()
        config.read_file(args.config)
        # Transforming values into integers
        args.n1 = int(config['ARGUMENTS']['n1'])
        args.n2 = int(config['ARGUMENTS']['n2'])
    main(args.n1, args.n2, args.output)

```
crontab -l
crontab -e  # for 5min
*/5 * * * * /fullpath_venv/python /fullpath2/cron.py -o /fullpath3/automate.log
```
__[crontab cheat sheet](https://crontab.guru/)__ 
```
* * * * * *
| | | | | |
| | | | | +-- Year (range: 1900-3000)
| | | | +---- Day of the Week (range: 1-7, 1 standing for Monday)
| | | +------ Month of the Year (range: 1-12)
| | +-------- Day of the Month (range: 1-31)
| +---------- Hour (range: 0-23)
+------------ Minute (range: 0-59)
Therefore, our line, */5 * * * * *, means every time the minute is divisible by 5, in all
hours, all days... all years.
Here are some examples:
30 15 * * * *   means "every day at 15:30"
30 * * * * *    means "every hour, at 30 minutes"
0,30 * * * * *  means "every hour, at 0 minutes and 30 minutes"
*/30 * * * * *  means "every half hour"
0 0 * * 1 *     means "every Monday at 00:00"
```
It is essential to specify full pathes everywhere in crontab. If any problem with crontab, you will receive email (e.g. no such file or directory)

`python ch02-cron.py -h` gives weird default for output `sys.stdout` -- __[change `formater_class` argument in `ArgumentParser`](https://docs.python.org/2/library/argparse.html#formatter-class)__

### Capturing errors and problems
Divide two numbers in `ch02-err.py`

In [ ]:
import argparse
import sys
def main(n1,n2,out):
    print(f'{n1} / {n2} = {n1/n2}',file=out)

if __name__ == '__main__':
    parser = argparse.ArgumentParser()
    parser.add_argument('-n1',type=int,default=1,help='numerator')
    parser.add_argument('-n2',type=int,default=1,help='denom')
    parser.add_argument('-o',dest='out',type=argparse.FileType('a'),default=sys.stdout,help='out')
    args = parser.parse_args()
    main(args.n1,args.n2,args.out)

In [ ]:
import logging
LOG_FORMAT = '%(asctime)s %(name)s %(levelname)s %(message)s'
LOG_LEVEL = logging.DEBUG

parser.add_argument('-l',type=str,dest='log',help='logfile')

if args.log:
    logging.basicConfig(format=LOG_FORMAT, level=LOG_LEVEL, filename=args.log)
else:
    logging.basicConfig(format=LOG_FORMAT, level=LOG_LEVEL)

try: 
    main(args.n1,args.n2,args.out)
    except Exception as exc:
        logging.exception("Error running task")
        print('There has been an error. Check the logs.')
        exit(1)


DEBUG, INFO, WARNING, ERROR, CRITICAL <br>
logging.warning() .info() .debug # only if level >= <br>
logging.exception() also provides stack trace <br>
__[documentation of the module](https://docs.python.org/3.8/library/logging.html)__ or
 __[tutorial](https://docs.python.org/3.8/howto/logging.html)__

